# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time 
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("Output/weather_data.csv")
city_weather_data.head()


,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Cidreira,BR,1614950872,-30.1811,-50.2056,75.97,80,90,14.05
1,1,Eilat,IL,1614950852,29.5581,34.9482,73.40,33,0,12.66
2,2,Maţāy,EG,1614950872,28.4190,30.7792,75.27,18,0,12.95
3,3,Cabo San Lucas,MX,1614950873,22.8909,-109.9124,63.00,88,75,3.44
4,4,Saskylakh,RU,1614950873,71.9167,114.0833,-30.39,95,0,8.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = city_weather_data[["Latitude", "Longitude"]]
humidity = city_weather_data["Humidity"]
max_humidity = humidity.max()

heat_map_figure = gmaps.figure(layout = {
        'width': '1200px',
        'height': '800px'})
heat_map_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = max_humidity, 
                                    point_radius = 3)

heat_map_figure.add_layer(heat_map_layer)

heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter cities: Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Adjust to your specifications but limit the number of rows returned by your API requests to a reasonable number.


filter_cities_df = city_weather_data.loc[(city_weather_data["Max Temperature"].astype(float) >= 70.0) & 
                                         (city_weather_data["Max Temperature"].astype(float) <= 80.0) & 
                                         (city_weather_data["Cloudiness"].astype(float) == 0.0) & 
                                         (city_weather_data["Wind Speed"].astype(float) < 10.0)]

filter_cities_df = filter_cities_df.dropna(how = "any")

del filter_cities_df["Unnamed: 0"]

filter_cities_df

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
5,Kruisfontein,ZA,1614950873,-34.0033,24.7314,75.99,65,0,7.00
35,Sūsangerd,IR,1614950876,31.5608,48.1831,71.60,35,0,6.91
125,Santiago,CL,1614950603,-33.4569,-70.6483,73.99,72,0,3.44
138,Rāmkola,IN,1614950888,26.9042,83.8411,77.58,16,0,8.05
175,Mount Isa,AU,1614950895,-20.7333,139.5000,71.60,24,0,8.05
305,Talā,EG,1614950912,30.6803,30.9429,74.86,20,0,6.13
345,Myanaung,MM,1614950917,18.2833,95.3167,79.68,20,0,5.84
512,Dingle,PH,1614950938,10.9995,122.6711,76.59,81,0,8.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# 1. Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# 2. Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.


hotel_df = pd.DataFrame(filter_cities_df, columns=["City", "Country", "Latitude", "Longitude"])
hotels = []

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params = {
        "location" : f"{latitude},{longitude}", 
        "radius" : 5000, 
        "types" : "hotel", 
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response =  requests.get(base_url, params = params)
    
    response_json = response.json()
    
    
    try:
        hotels.append(response_json['results'][0]['name'])
    except IndexError:
        hotels.append(np.nan)
    
    time.sleep(1)

In [7]:
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
5,Kruisfontein,ZA,-34.0033,24.7314,Humansdorp
35,Sūsangerd,IR,31.5608,48.1831,Susangerd
125,Santiago,CL,-33.4569,-70.6483,Santiago
138,Rāmkola,IN,26.9042,83.8411,Ramkola
175,Mount Isa,AU,-20.7333,139.5000,Mount Isa
305,Talā,EG,30.6803,30.9429,Tala
345,Myanaung,MM,18.2833,95.3167,Myanaung
512,Dingle,PH,10.9995,122.6711,Pototan


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer on top of heat map

markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
heat_map_figure.add_layer(markers)

# Display Map
heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))